In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.getcwd()

In [ ]:
path="/content/drive/MyDrive/Colab_Notebooks/panda_grasp/"
os.chdir(path)
os.getcwd()

In [ ]:
!ls

In [ ]:
cd Environment/

In [ ]:
pip install stable-baselines3[extra]

In [ ]:
pip install gym

In [ ]:
pip install pybullet

In [1]:
import argparse
import difflib
import os
import uuid

import gym
import numpy as np
import seaborn
import torch as th
from stable_baselines3.common.utils import set_random_seed
import task
from utils.exp_manager import ExperimentManager
from utils.utils import ALGOS, StoreDict

seaborn.set()


In [2]:
def train(args=None):

    # Check if the selected environment is valid
    # If it could not be found, suggest the closest match
    registered_envs = set(gym.envs.registry.env_specs.keys())
    if args.env not in registered_envs:
        try:
            closest_match = difflib.get_close_matches(
                args.env, registered_envs, n=1)[0]
        except IndexError:
            closest_match = "'no close match found...'"
        raise ValueError(
            f"{args.env} not found in gym registry, you maybe meant {closest_match}?")

    # If no specific seed is selected, choose a random one
    if args.seed < 0:
        args.seed = np.random.randint(2 ** 32 - 1, dtype="int64").item()

    # Set the random seed across platforms
    set_random_seed(args.seed)

    # Setting num threads to 1 makes things run faster on cpu
    if args.num_threads > 0:
        if args.verbose > 1:
            print(f"Setting torch.num_threads to {args.num_threads}")
        th.set_num_threads(args.num_threads)

    # Verify that pre-trained agent exists before continuing to train it
    if args.trained_agent != "":
        assert args.trained_agent.endswith(".zip") and os.path.isfile(
            args.trained_agent
        ), "The trained_agent must be a valid path to a .zip file"

    # If enabled, ensure that the run has a unique ID
    uuid_str = f"_{uuid.uuid4()}" if args.uuid else ""

    print("=" * 10, args.env, "=" * 10)
    print(f"Seed: {args.seed}")

    exp_manager = ExperimentManager(
        args,
        args.algo,
        args.env,
        args.log_folder,
        args.tensorboard_log,
        args.n_timesteps,
        args.eval_freq,
        args.eval_episodes,
        args.save_freq,
        args.hyperparams,
        args.env_kwargs,
        args.trained_agent,
        args.optimize_hyperparameters,
        args.storage,
        args.study_name,
        args.n_trials,
        args.n_jobs,
        args.sampler,
        args.pruner,
        n_startup_trials=args.n_startup_trials,
        n_evaluations=args.n_evaluations,
        truncate_last_trajectory=args.truncate_last_trajectory,
        uuid_str=uuid_str,
        seed=args.seed,
        log_interval=args.log_interval,
        save_replay_buffer=args.save_replay_buffer,
        
        verbose=args.verbose,
        vec_env_type=args.vec_env,
    )

    # Prepare experiment and launch hyperparameter optimization if needed
    model = exp_manager.setup_experiment()

    if args.optimize_hyperparameters:
        exp_manager.hyperparameters_optimization()
    else:
        exp_manager.learn(model)
        exp_manager.save_trained_model(model)

In [3]:
parser = argparse.ArgumentParser()

# Environment and its parameters
parser.add_argument("--env", type=str,
                        default="PandaTouchEnv_color-v0",
                        help="environment ID")
parser.add_argument("--env-kwargs", type=str, nargs="+", action=StoreDict,
                        help="Optional keyword argument to pass to the env constructor")
parser.add_argument("--vec-env", type=str, choices=["dummy", "subproc"],
                        default="dummy",
                        help="VecEnv type")

# Algorithm
parser.add_argument("--algo", type=str, choices=list(ALGOS.keys()), required=False,
                        default="sac", help="RL Algorithm")
parser.add_argument("-params", "--hyperparams", type=str, nargs="+", action=StoreDict,
                        help="Overwrite hyperparameter (e.g. learning_rate:0.01 train_freq:10)")
parser.add_argument("--num-threads", type=int,
                        default=-1,
                        help="Number of threads for PyTorch (-1 to use default)")

# Training duration
parser.add_argument("-n", "--n-timesteps", type=int,
                        default=50000,
                        help="Overwrite the number of timesteps")

# Continue training an already trained agent
parser.add_argument("-i", "--trained-agent", type=str,
                        default="",
                        help="Path to a pretrained agent to continue training")

# Random seed
parser.add_argument("--seed", type=int,
                        default=42,
                        help="Random generator seed")

# Saving of model
parser.add_argument("--save-freq", type=int,
                        default=1000,
                        help="Save the model every n steps (if negative, no checkpoint)")
parser.add_argument("--save-replay-buffer", action="store_true",
                        default=False,
                        help="Save the replay buffer too (when applicable)")

# Pre-load a replay buffer and start training on it
parser.add_argument("--preload-replay-buffer", type=str,
                        default="model",
                        help="Path to a replay buffer that should be preloaded before starting the training process")

# Logging
parser.add_argument("-f", "--log-folder", type=str,
                        default="logs",
                        help="Log folder")
parser.add_argument("-tb", "--tensorboard-log", type=str,
                        default="tensorboard_logs",
                        help="Tensorboard log dir")
parser.add_argument("--log-interval", type=int,
                        default=-1,
                        help="Override log interval (default: -1, no change)")
parser.add_argument("-uuid", "--uuid", action="store_true",
                        default=False,
                        help="Ensure that the run has a unique ID")

# Hyperparameter optimization
parser.add_argument("-optimize", "--optimize-hyperparameters", action="store_true",
                        default=False,
                        help="Run hyperparameters search")
parser.add_argument("--sampler", type=str, choices=["random", "tpe", "skopt"],
                        default="tpe",
                        help="Sampler to use when optimizing hyperparameters")
parser.add_argument("--pruner", type=str, choices=["halving", "median", "none"],
                        default="median",
                        help="Pruner to use when optimizing hyperparameters")
parser.add_argument("--n-trials", type=int,
                        default=10,
                        help="Number of trials for optimizing hyperparameters")
parser.add_argument("--n-startup-trials", type=int,
                        default=5,
                        help="Number of trials before using optuna sampler")
parser.add_argument("--n-evaluations", type=int,
                        default=2,
                        help="Number of evaluations for hyperparameter optimization")
parser.add_argument("--n-jobs", type=int,
                        default=1,
                        help="Number of parallel jobs when optimizing hyperparameters")
parser.add_argument("--storage", type=str,
                        default=None,
                        help="Database storage path if distributed optimization should be used")
parser.add_argument("--study-name", type=str,
                        default=None,
                        help="Study name for distributed optimization")

# Evaluation
parser.add_argument("--eval-freq", type=int,
                        default=-1,
                        help="Evaluate the agent every n steps (if negative, no evaluation)")
parser.add_argument("--eval-episodes", type=int,
                        default=10,
                        help="Number of episodes to use for evaluation")

# Verbosity
parser.add_argument("--verbose", type=int,
                        default=1,
                        help="Verbose mode (0: no output, 1: INFO)")

# HER specifics
parser.add_argument(
        "--truncate-last-trajectory",
        help="When using HER with online sampling the last trajectory "
        "in the replay buffer will be truncated after reloading the replay buffer.",
        default=True,
        type=bool,
)
args, unknown = parser.parse_known_args()

In [4]:
print(args.preload_replay_buffer)
print(args.algo)

model
sac


In [5]:
train(args=args)

========== PandaTouchEnv_color-v0 ==========
Seed: 42
OrderedDict([('batch_size', 32),
             ('buffer_size', 25000),
             ('ent_coef', 'auto_0.1'),
             ('env_wrapper',
              ['task.wrapper.ProcessFrame84',
               'task.wrapper.MoveConstraint',
               {'task.wrapper.TimeLimit': {'max_episode_steps': 400}}]),
             ('gamma', 0.95),
             ('gradient_steps', 1),
             ('learning_rate', 'lin_0.0003'),
             ('learning_starts', 0),
             ('n_timesteps', 50000),
             ('noise_std', 0.025),
             ('noise_type', 'normal'),
             ('optimize_memory_usage', True),
             ('policy', 'CnnPolicy'),
             ('policy_kwargs', {'n_critics': 2, 'net_arch': [128, 128]}),
             ('target_entropy', 'auto'),
             ('tau', 0.01),
             ('train_freq', 1)])
Using 1 environments
Overwriting n_timesteps with n=50000


ic| self.object_object1_id: 3


Wrapping into a VecTransposeImage
Applying normal noise with std 0.025
Using cuda device


ic| '============reset=================='


Log path: logs/sac/PandaTouchEnv_color-v0_16
Logging to tensorboard_logs/PandaTouchEnv_color-v0/SAC_13


...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 81.9     |
|    ep_rew_mean     | 1        |
| time/              |          |
|    episodes        | 132      |
|    fps             | 3        |
|    time_elapsed    | 10498    |
|    total timesteps | 41667    |
| train/             |          |
|    actor_loss      | -0.0143  |
|    critic_loss     | 2.05e-06 |
|    ent_coef        | 7.2e-05  |
|    ent_coef_loss   | -56.4    |
|    learning_rate   | 5e-05    |
|    n_updates       | 41666    |
---------------------------------


ic| '============reset=================='
ic| '============reset=================='
ic| '============reset=================='
ic| '============reset=================='


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 81.9     |
|    ep_rew_mean     | 1        |
| time/              |          |
|    episodes        | 136      |
|    fps             | 3        |
|    time_elapsed    | 10962    |
|    total timesteps | 43267    |
| train/             |          |
|    actor_loss      | -0.0141  |
|    critic_loss     | 2.12e-08 |
|    ent_coef        | 6.7e-05  |
|    ent_coef_loss   | -58.5    |
|    learning_rate   | 4.04e-05 |
|    n_updates       | 43266    |
---------------------------------


ic| '============reset=================='


Saving to logs/sac/PandaTouchEnv_color-v0_16


In [ ]:
%tensorboard --logdir ./tensorboard_logs/PandaGraspEnv_color-v0/

SyntaxError: invalid syntax (<ipython-input-4-f56d0c02a80c>, line 1)